In [1]:
import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import os
import random

# .env 파일에서 OpenAI API 키 로드
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
# 태그 목록
tags = [
    "병원", "육아 상담"
]

# 조건 설정
ages = ["20대 후반", "30대 초반", "30대 후반"]
experiences = ["초보", "둘째를 키우는"]
genders = ["엄마", "아빠"]
child_ages = ["0세", "1세", "2세"]
seasons = ["봄", "여름", "가을", "겨울"]
special_situations = ["일반", "쌍둥이", "장애아", "다문화 가정"]

# 비율 설정
age_ratio = [1, 1, 1]
experience_ratio = [1, 1]
gender_ratio = [6, 4]
child_age_ratio = [1, 1, 1]
season_ratio = [1, 1, 1, 1]
special_situation_ratio = [7, 1, 1, 1]

def generate_prompt(tag, age, experience, gender, child_age, season, special_situation):
    base_prompt = (
        f"{age} {experience} {gender}가 {child_age} 자녀를 키우고 있습니다. "
        f"현재 계절은 {season}입니다. "
    )
    
    if special_situation != "일반":
        base_prompt += f"이 가정은 {special_situation} 상황입니다. "
    
    if tag == "병원":
        base_prompt += (
            f"{child_age} 자녀를 키우는 {gender}의 입장에서, 병원 위치, 병원 정보, "
            f"또는 급한 병원 방문이 필요한 상황에 대한 질문을 하나 생성해주세요. "
            f"질문에는 반드시 '병원' 키워드가 포함되어야 합니다. "
            f"육아 상담 태그와 헷갈리지 않는 질문이어야 합니다."
        )
    elif tag == "육아 상담":
        base_prompt += (
            f"{child_age} 자녀를 키우는 {gender}의 입장에서, 아이의 행동이나 발달에 대한 조언을 구하는 질문을 하나 생성해주세요. "
            f"질문은 급하지 않은 일상적인 육아 고민이어야 합니다. "
            f"병원 태그와 헷갈리지 않는 질문이어야 합니다."
        )
    
    base_prompt += (
        f"질문은 30자 내외로 작성하고, 최근에 겪은 특정 상황이나 문제점을 포함하며, "
        f"실제 부모가 사용할 법한 자연스러운 표현으로 작성해주세요. "
    )
    
    return base_prompt

In [3]:
# 데이터 프레임 초기화
data = []

# 각 태그별로 100개의 질문 생성
for tag in tags:
    for _ in tqdm(range(100), desc=f"Generating questions for tag: {tag}"):
        while True:
            # 각 비율에 따라 무작위 선택
            age = random.choices(ages, weights=age_ratio, k=1)[0]
            experience = random.choices(experiences, weights=experience_ratio, k=1)[0]
            gender = random.choices(genders, weights=gender_ratio, k=1)[0]
            child_age = random.choices(child_ages, weights=child_age_ratio, k=1)[0]
            season = random.choices(seasons, weights=season_ratio, k=1)[0]
            special_situation = random.choices(special_situations, weights=special_situation_ratio, k=1)[0]
            
            # 프롬프트 생성
            prompt = generate_prompt(tag, age, experience, gender, child_age, season, special_situation)
            
            # GPT-3.5-turbo API 호출
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100
            )
            
            # 생성된 질문 추출
            question = response['choices'][0]['message']['content'].strip()
            
            # 태그에 맞는 질문인지 확인
            if tag == "병원" and "병원" in question:
                break
            elif tag == "육아 상담" and "병원" not in question:
                break
        
        # 데이터 추가
        data.append({
            "tag": tag,
            "age": age,
            "experience": experience,
            "gender": gender,
            "child_age": child_age,
            "season": season,
            "special_situation": special_situation,
            "question": question
        })

# 데이터 프레임 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv("generated_questions.csv", index=False)

print("Questions generation complete. Saved to 'generated_questions.csv'.")

Generating questions for tag: 병원:   0%|          | 0/100 [00:00<?, ?it/s]

Generating questions for tag: 육아 상담: 100%|██████████| 100/100 [01:51<00:00,  1.12s/it]

Questions generation complete. Saved to 'generated_questions.csv'.


In [5]:
import pandas as pd
import random

# processed_data.csv에서 '병원' 또는 '의학 정보' 태그 삭제
processed_data = pd.read_csv('processed_data.csv')
processed_data = processed_data[~processed_data['tag'].isin(['병원', '의학 정보'])]

# qq.csv에서 '병원' 태그 행 추출
qq_data = pd.read_csv('qq.csv')
qq_filtered = qq_data[qq_data['tag'] == '병원']

# aa.csv에서 question 열에서 랜덤으로 100개 추출 및 태그 추가
aa_data = pd.read_csv('aa.csv')
aa_sampled = aa_data.sample(n=100, random_state=1)  # 랜덤 시드로 재현 가능하게 설정
aa_sampled['tag'] = '육아 의학 상담'

# 최종 데이터프레임 합치기
final_data = pd.concat([processed_data, qq_filtered, aa_sampled[['tag', 'question']]], ignore_index=True)

# CSV 파일로 저장
final_data.to_csv('merged_data.csv', index=False)

print("CSV 파일이 성공적으로 생성되었습니다. 'merged_data.csv' 파일을 확인해주세요.")


CSV 파일이 성공적으로 생성되었습니다. 'merged_data.csv' 파일을 확인해주세요.
